# News Search and Summarizer

This notebook searches for latest news and provides summaries.

In [ ]:
# Install required packages (run once)
#%pip install openai feedparser requests python-dotenv

In [ ]:
# import feedparser
# import requests
# from openai import OpenAI
# from datetime import datetime

# %pip install  dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]
Note: you may need to restart the kernel to use updated packages.


In [23]:
import feedparser
import requests
from openai import OpenAI
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [28]:
# Initialize OpenAI client
# API key is loaded from .env file
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file. Please add it to your .env file.")

client = OpenAI(api_key=api_key)

ValueError: OPENAI_API_KEY not found in .env file. Please add it to your .env file.

## 1. Search Latest News

Using Google News RSS feed (free, no API key needed)

In [11]:
def fetch_latest_news(topic='world', num_articles=5):
    """
    Fetch latest news from Google News RSS feed
    
    Args:
        topic: News topic (e.g., 'world', 'technology', 'business')
        num_articles: Number of articles to fetch
    """
    url = f'https://news.google.com/rss/search?q={topic}&hl=en-US&gl=US&ceid=US:en'
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:num_articles]:
        article = {
            'title': entry.title,
            'link': entry.link,
            'published': entry.published if 'published' in entry else 'N/A',
            'summary': entry.summary if 'summary' in entry else entry.title
        }
        articles.append(article)
    
    return articles

In [12]:
# Fetch latest news
topic = 'technology'  # Change to any topic you want
news_articles = fetch_latest_news(topic=topic, num_articles=5)

print(f"Latest {topic} news:\n")
for i, article in enumerate(news_articles, 1):
    print(f"{i}. {article['title']}")
    print(f"   Published: {article['published']}")
    print(f"   Link: {article['link']}")
    print()

Latest technology news:

1. Carney signs onto AI and technology partnership with India and Australia - CBC
   Published: Sat, 22 Nov 2025 15:34:53 GMT
   Link: https://news.google.com/rss/articles/CBMihwFBVV95cUxNeUNTMkhOc2JibXRiRkRWXzJvU2JnaF9sM2tZSUlzM0dLQnNidmNoNE02M3pmX0JJYXRjNzE4QXdGNnZ3M1lLSGNDZDJ3ZE1CWkpFRXhsX3FfTTFvVFNmVmpwTXpycndyUTlkNnZTN09MaXpwQlNYaV9uVzJGZ3I4UjJWZzBhV3c?oc=5

2. India, Australia and Canada to form new trilateral technology and innovation alliance - The Economic Times
   Published: Sat, 22 Nov 2025 16:01:13 GMT
   Link: https://news.google.com/rss/articles/CBMi6wFBVV95cUxOY3N1WjNkWjlSTHlyRGNDaGFWYXEyUkJzdU1FemxHTDBLbnVGcV9JNVNuS2U0aDE4LWtQY2ZyMnYwOXlkVXU5U2lRUkp0MFYwd2JVVUNFakt4QXlGTDBRbUVjdkJCcThPa2UwVjNkVTM0eHVyYklFalRHalJWRVR6OUJhLUJlX2paYTFLYWVjS2J5dC02Mm1PNUNuaU5nZ1JaZXZ1cmNoWGtWMVZxUXByVm1jWExmOVRJWGx2V01HR2JxN2pka2NaWUloNjBXdXJxUkRNVjNKVTNPRGdYZmYtcFZOU1lPM1JmSXcw0gHwAUFVX3lxTE5FVDJld1VINzYwZWFwOWVENkhXNFA2eGhqTDVaTkZ2ZFNDYl9YNjFlbjNwSkd6ZzdQZTdNUGxrS

## 2. Summarize News with OpenAI

In [14]:
def summarize_news_with_openai(articles):
    """
    Create a summary of multiple news articles using OpenAI
    """
    # Combine all article titles and summaries
    news_text = "\n\n".join([
        f"Article {i+1}: {article['title']}\n{article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    # Call OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Use gpt-4o or gpt-3.5-turbo based on your preference
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes news articles concisely."},
            {"role": "user", "content": f"Please provide a brief summary of these news articles:\n\n{news_text}"}
        ],
        max_tokens=500,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [15]:
# Generate summary
summary = summarize_news_with_openai(news_articles)

print("="*60)
print("NEWS SUMMARY")
print("="*60)
print(summary)
print("="*60)

NEWS SUMMARY
**Article 1: Carney signs onto AI and technology partnership with India and Australia - CBC**  
Mark Carney has officially joined a partnership focused on artificial intelligence and technology collaboration between Canada, India, and Australia. This initiative aims to strengthen ties and promote innovation in the tech sector among the three nations.

**Article 2: India, Australia and Canada to form new trilateral technology and innovation alliance - The Economic Times**  
India, Australia, and Canada are establishing a new trilateral alliance that will focus on technology and innovation. This collaboration is expected to bolster technological advancements and foster innovation across the three countries.

**Article 3: PM Modi meets Mark Carney, Albanese; announces Australia-Canada-India Technology & Innovation Partnership - livemint.com**  
During a meeting with Mark Carney and Australian Prime Minister Anthony Albanese, Indian Prime Minister Narendra Modi announced the f

## 3. All-in-One Function

In [18]:
def search_and_summarize_news(topic='world', num_articles=5):
    """
    Search for news and generate a summary in one function
    """
    print(f"Fetching latest news about '{topic}'...\n")
    articles = fetch_latest_news(topic=topic, num_articles=num_articles)
    
    print(f"Found {len(articles)} articles:\n")
    for i, article in enumerate(articles, 1):
        print(f"{i}. {article['title']}")
    
    print("\nGenerating summary...\n")
    summary = summarize_news_with_openai(articles)
    
    print("="*60)
    print("SUMMARY")
    print("="*60)
    print(summary)
    print("="*60)
    
    return articles, summary

In [20]:
# Quick usage example
#articles, summary = search_and_summarize_news(topic='artificial intelligence', num_articles=5)